# 政策法规提取和预处理

政策法规文件来自**北大法宝**。为了便于后续的知识库构建和问答系统开发，需要对这些文件进行预处理，包括解压缩、文本提取和清洗等步骤。

In [ ]:
import os
import shutil
import zipfile

def unzip_all(src_dir: str, dst_dir: str):
    """
    解压src_dir目录下的所有zip文件，并把内容移动到dst_dir目录
    解决中文文件名乱码问题
    """

    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir, exist_ok=True)

    for file in os.listdir(src_dir):
        if file.lower().endswith(".zip"):
            zip_path = os.path.join(src_dir, file)
            print(f"正在解压: {zip_path}")

            try:
                with zipfile.ZipFile(zip_path, 'r') as zf:
                    for info in zf.infolist():
                        # 解决中文乱码
                        try:
                            filename = info.filename.encode('cp437').decode('utf-8')
                        except UnicodeDecodeError:
                            filename = info.filename.encode('cp437').decode('gbk', errors='ignore')

                        # 目标路径
                        extracted_path = os.path.join(dst_dir, filename)

                        # 如果是目录，创建
                        if info.is_dir():
                            os.makedirs(extracted_path, exist_ok=True)
                        else:
                            # 确保父目录存在
                            os.makedirs(os.path.dirname(extracted_path), exist_ok=True)
                            with zf.open(info) as source, open(extracted_path, "wb") as target:
                                shutil.copyfileobj(source, target)

                print(f"完成解压: {file}")

            except Exception as e:
                print(f"解压失败 {file}: {e}")


In [ ]:
source_directory = "./政策zip"
target_directory = "./政策"
unzip_all(source_directory, target_directory)

In [ ]:
# -*- coding: utf-8 -*-
"""
HTML到Markdown转换器
该脚本用于解析特定格式的HTML文件，提取元数据和正文内容，
并将内容转换为Markdown格式。
"""

from bs4 import BeautifulSoup
from markdownify import markdownify as md


def parse_html_to_structured_data(html_string: str) -> dict:
    """
    解析给定的HTML字符串，提取元数据和转换为Markdown的正文内容。
    Args:
        html_string: 包含法律文档的HTML字符串。

    Returns:
        一个包含解析后数据的字典。
    """
    
    soup = BeautifulSoup(html_string, 'lxml')
    result_data = {}
    
    field_mapping = {
        '制定机关': 'issuing_authority',
        '发文字号': 'document_number',
        '公布日期': 'publish_date',
        '施行日期': 'effective_date',
        '时效性': 'timeliness',
        '效力位阶': 'effectiveness_level',
        '法规类别': 'category'
    }

    # --- 提取元数据 ---
    title_tag = soup.find('h2', class_='title')
    if title_tag:
        result_data['title'] = title_tag.get_text(strip=True)

    fields_container = soup.find('div', class_='fields')
    if fields_container:
        for li in fields_container.find_all('li'):
            box = li.find('div', class_='box')
            if not box: continue
            strong_tag = box.find('strong')
            if not strong_tag: continue
            field_name_cn = strong_tag.get_text(strip=True).replace('：', '')
            field_name_en = field_mapping.get(field_name_cn)
            
            if field_name_en:
                if field_name_en == 'category':
                    values = [a.get_text(strip=True) for a in box.find_all('a')]
                    result_data[field_name_en] = ' '.join(values)
                else:
                    value_tag = box.find('a')
                    if value_tag:
                        result_data[field_name_en] = value_tag.get_text(strip=True)
                    else:
                        full_text = box.get_text(strip=True)
                        value = full_text.replace(strong_tag.get_text(strip=True), '', 1).strip()
                        result_data[field_name_en] = value

    # --- 处理正文内容并转换为Markdown ---
    content_div = soup.find('div', id='divFullText')
    
    if content_div:
        # 在转换为Markdown之前，遍历所有<a>标签
        # 并用标签内的文本替换掉整个<a>标签。
        # 例如: <a href="...">文本</a> 会直接变成 "文本"
        for a_tag in content_div.find_all('a'):
            a_tag.replace_with(a_tag.get_text())
        # ----------------------------------------------------
            
        # 现在 content_div 里的HTML已经没有<a>标签了，可以直接转换
        markdown_content = md(str(content_div), heading_style="ATX", strip=['br'])

        lines = markdown_content.splitlines()
        cleaned_lines = []
        for line in lines:
            # 移除可能由markdownify产生的多余转义或格式问题
            cleaned_line = line.rstrip()
            # 移除由<p><br></p>等产生的空行 (如果markdownify未处理好)
            if not (cleaned_line.isspace() or cleaned_line == "") or (cleaned_lines and not (cleaned_lines[-1].isspace() or cleaned_lines[-1] == "")):
                 cleaned_lines.append(cleaned_line)
        
        # 进一步清理，移除开头和结尾的空行
        while cleaned_lines and (cleaned_lines[0].isspace() or cleaned_lines[0] == ""):
            cleaned_lines.pop(0)
        while cleaned_lines and (cleaned_lines[-1].isspace() or cleaned_lines[-1] == ""):
            cleaned_lines.pop()

        markdown_content = "\n".join(cleaned_lines)
        
        result_data['content_markdown'] = markdown_content.strip().replace('\u3000', '')

    return result_data

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# 遍历文件夹下的所有文件
input_folder = target_directory
all_parsed_items = []

for file_name in tqdm(os.listdir(input_folder)):
    if file_name.endswith('.html'):
        file_path = os.path.join(input_folder, file_name)
        try:
            # 读取HTML文件内容
            with open(file_path, 'r', encoding='utf-8') as f:
                html_content = f.read()

            # 解析HTML内容
            parsed_items = parse_html_to_structured_data(html_content)
            all_parsed_items.append(parsed_items)
        except Exception as e:
            print(f"处理文件 '{file_name}' 时发生错误: {e}")
            break

df = pd.DataFrame(all_parsed_items)
df

In [ ]:
# --- 数据处理和按年统计 ---
# 1. 将 'publish_date' 列转换为字符串类型（以防万一）
df['publish_date'] = df['publish_date'].astype(str)
df['effective_date'] = df['effective_date'].astype(str)

# 2. 定义一个函数来解析不规则的日期字符串
def parse_incomplete_date(date_str):
    """尝试将 'YYYY.MM.DD', 'YYYY.MM', 或 'YYYY' 格式的字符串解析为 datetime 对象"""
    formats = ['%Y.%m.%d', '%Y.%m', '%Y'] # 尝试的日期格式列表
    for fmt in formats:
        try:
            # 对于只有年份的情况，pandas.to_datetime 默认会将月份和日期设为 1月1日
            return pd.to_datetime(date_str, format=fmt, errors='raise')
        except ValueError:
            continue # 如果当前格式不匹配，则尝试下一个格式
    # 如果所有格式都失败，则返回 NaT (Not a Time)
    return pd.NaT

# 3. 应用函数解析日期列
df['publish_date'] = df['publish_date'].apply(parse_incomplete_date)
df['effective_date'] = df['effective_date'].apply(parse_incomplete_date)

In [ ]:
# 保存数据
df.to_parquet("政策法规.parquet")